# Find Similar Sequences from Refseq blast of the Diamond X-ray screen protein

In [ ]:
from pathlib import Path
from pymol import cmd
import os
import re

In [ ]:
# Going to be working in this directory from now on to keep things cleaner
local_path = Path("/Users/choderalab/temp_storage/arborvirus/")

## Get just the protein from the X-ray screen

In [ ]:
structure_name = 'zikv_ligands'
cmd.load(local_path / "ZIKV_NS2B3_ligands.pdb", structure_name)
cmd.select("zikv_protein", structure_name + " and polymer.protein")
#cmd.save(local_path / "protein.pdb", "protein")

In [ ]:
# Save the fasta of that structure
cmd.save(local_path/"zikv_ns2b3.fasta", "zikv_protein and chain A+B")

## Refseq Blast the protein fasta

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO

In [ ]:
# Define sequence to compare
sequence = ""

# Read in the sequence to compare
# https://www.biostars.org/p/710/
fasta_file = local_path/"zikv_ns2b3.fasta"
fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')
for fasta in fasta_sequences:
    name, seq = fasta.id, str(fasta.seq)
    # Separate the chains with : 
    sequence += seq + ":"
# Remove the last : 
sequence = sequence[:-1]

In [ ]:
# Define parameters
blast_program = "blastp"
database = "refseq_protein"

In [ ]:
# Perform BLAST query
result_handle = NCBIWWW.qblast(blast_program, database, sequence, format_type="XML",hitlist_size = 150)

In [ ]:
# Save result to file
with open(local_path/"zikv_blast_both_chain_results.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

## Filter the blast results by organisms of interest
https://github.com/biopython/biopython/blob/master/Bio/Blast/NCBIXML.py

In [ ]:
# Find the organisms interested in 
# Define a set of organisms you are interested in
organisms_of_interest = {"West Nile virus", "dengue virus type 4","dengue virus type 3","dengue virus type 2","dengue virus type 1"}

# Store hits from organisms of interest
hits_from_interesting_organisms = []
# Species indicated between brackets
species_pattern = r'\[(.*?)\]'

# Iterate over blast records
with open(local_path/"zikv_blast_both_chain_results.xml", 'r') as file:
    # Read the contents of the file
    blast_records = NCBIXML.parse(file)
    # Print out the BLAST results
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            title =  alignment.title
            species = re.search(species_pattern, title).group(1)
            # Only keep the output if the species is within organisms_of_interest
            if species in organisms_of_interest:
                for hsp in alignment.hsps:
                    # Save the found sequence title, sequence, and E-value of this match
                    hits_from_interesting_organisms.append((alignment.title,hsp.sbjct,hsp.expect))


### Save structure of blast result as list of tuples
#### Tuple structure: (title of blast search(with organism name and id), sequence from the search of the match, E-value of the blast)

## This may be an issue because the sequences do not have any distinction between the different chains, so unsure how this will be resolved in colabfold

## Remove all duplicate sequences from the file of interest and truncated

In [ ]:
# Delete the duplicate sequences from list of all the organism sequences
# Get rid of sequences that are too short
# Get rid of sequences that are not matched with enough 
# Get rid of blanks (---) within the sequence if present (This is needed for colabfold)
# https://biopython.org/wiki/Sequence_Cleaner
# E-value: https://biology.stackexchange.com/questions/19338/what-e-value-blast-cut-off-is-best
def sequence_cleaner(seq_list, handle_blank = True, min_length=0,e_value_threshold=1):
    # Create our hash table to add the sequences
    sequences = {}

    # See if the sequence are truncated
    for seq_info in seq_list:
        sequence = str(seq_info[1]).upper()
        # Handle the blanks (----) in the sequence
        if handle_blank:
            blanks = r'-+'
            # Remove all blanks so real length
            sequence = re.sub(blanks,'',sequence)
        
        if (
            len(sequence) >= min_length
            and e_value_threshold > seq_info[2]
        ):
            # check if in hash table, the sequence and its id are going to be in the hash
            if sequence not in sequences:
                sequences[sequence] = seq_info[0]
            # If it is already in the hash table, just going to skip
        
    return sequences


clean_sequences = sequence_cleaner(hits_from_interesting_organisms,min_length=100)


In [ ]:
# Create to save the cleaned fasta results
with open(local_path /"clear_seqs_found.txt", "w+") as output_file:
    # Just read the hash table and write on the file as a fasta format
    for sequence in clean_sequences:
        output_file.write(">" + clean_sequences[sequence] + "\n" + sequence + "\n")

print("CLEAN!!!\nPlease check clear_seqs_found.txt")

In [ ]:
# Create to save the cleaned fasta results
clean_blank_sequences = sequence_cleaner(hits_from_interesting_organisms,handle_blank = False, min_length=100)

with open(local_path /"clear_blank_seqs_found.txt", "w+") as output_file:
    # Just read the hash table and write on the file as a fasta format
    for sequence in clean_blank_sequences:
        output_file.write(">" + clean_blank_sequences[sequence] + "\n" + sequence + "\n")

print("CLEAN!!!\nPlease check clear_seqs_found.txt")

### Save all the sequences that are found from species of interest into a file
#### interest_fasta.txt

## Put into csv for Colabfold

In [ ]:
# List to put in csv
id_seqs = []

regex = r'ref\|(\w+\.\d+)\|'
for sequence in clean_sequences:
    text = clean_sequences[sequence]
    match = re.match(regex, text)
    id_seqs.append((match.group(1)+"_{}",sequence))

In [ ]:
import csv
# Output the previous thing to csv format
# Specify the file path
file_path = "arborviruses_full.csv"

# Write data to CSV file
with open(local_path/file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'sequence'])  # Write header
    writer.writerows(id_seqs)  # Write data rows

## Now Running Colabfold from lilac